# SQL DEMO

A quick demo on how to perform SQL queries in Python and compare them to dataframe functions.

### Table of Contents

#### 1. [Load Data](#LoadData)
#### 2. [Analysis](#Analysis)
#### 3. [Modeling](#Modeling)
#### 4. [Evaluate](#Evaluate)

## LoadData <a id='LoadData'></a>
I'll use this dataset from Kaggle: [LINK](https://www.kaggle.com/datasets/leandrojnr/applestore-database). 
This example will use pandas to load data from csv to dataframes. 

### Import Python Libraries

In [71]:
import pandas as pd
import sqlite3

### Create the DB connection
You can use a temporary connection in memory to store data and practice.



In [72]:
conn = sqlite3.connect(':memory:') 
cur = conn.cursor()

### Define file location

In [73]:
location = "X:/DB/Apple Store Dataset"
input0 = f"{location}/Copy of AppleStore.csv"
input1 = f"{location}/Copy of appleStore_description1.csv"
input2 = f"{location}/Copy of appleStore_description2.csv"
input3 = f"{location}/Copy of appleStore_description3.csv"
input4 = f"{location}/Copy of appleStore_description4.csv"

### Use Pandas to read csv to dataframe

In [74]:
apps = pd.read_csv(input0)
df1 = pd.read_csv(input1)
df2 = pd.read_csv(input2)
df3 = pd.read_csv(input3)
df4 = pd.read_csv(input4)

### Union all description tables into 1 table 

In [75]:
df_desc = pd.concat([df1, df2, df3, df4], ignore_index=True)

### Use to_sql command to create tables in the DB

In [76]:
apps.to_sql(name='Apps', con=conn, if_exists='replace')
df1.to_sql(name='descriptionfile1', con=conn, if_exists='replace')
df2.to_sql(name='descriptionfile2', con=conn, if_exists='replace')
df3.to_sql(name='descriptionfile3', con=conn, if_exists='replace')
df4.to_sql(name='descriptionfile4', con=conn, if_exists='replace')

2199

## Analysis <a id='Analysis'></a>

### Print the number of elements in the dfs

In [77]:
num_rows_apps = len(apps)
print (f"Apps Table Row Count: {num_rows_apps}")

num_rows_desc = len(df_desc)
print (f"Description Table Row Count: {num_rows_desc}")

Apps Table Row Count: 7197
Description Table Row Count: 7197


### Print header and first 2 rows to view column names

#### Apps df ex: 

In [78]:
print(apps.head(2))

   Unnamed: 0         id                 track_name  size_bytes currency  \
0           1  281656475            PAC-MAN Premium   100788224      USD   
1           2  281796108  Evernote - stay organized   158578688      USD   

   price  rating_count_tot  rating_count_ver  user_rating  user_rating_ver  \
0   3.99             21292                26          4.0              4.5   
1   0.00            161065                26          4.0              3.5   

     ver cont_rating   prime_genre  sup_devices_num  ipadSc_urls_num  \
0  6.3.5          4+         Games               38                5   
1  8.2.2          4+  Productivity               37                5   

   lang_num  vpp_lic  
0        10        1  
1        23        1  


#### Descriptions df ex: 

In [79]:
print(df_desc.head(2))

          id                 track_name  size_bytes  \
0  281656475            PAC-MAN Premium   100788224   
1  281796108  Evernote - stay organized   158578688   

                                            app_desc  
0  SAVE 20%. now only $3.99 for a limited time!\n...  
1  Let Evernote change the way you organize your ...  


### Column names

In [80]:
print(f'Description DF Columns:',  df_desc.columns.tolist()) 
print('')
print(f'Apps DF Columns:',  apps.columns.tolist())

Description DF Columns: ['id', 'track_name', 'size_bytes', 'app_desc']

Apps DF Columns: ['Unnamed: 0', 'id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices_num', 'ipadSc_urls_num', 'lang_num', 'vpp_lic']


### Use this to see what dfs are in memory

In [81]:
%whos DataFrame

Variable   Type         Data/Info
---------------------------------
apps       DataFrame          Unnamed: 0         <...>n[7197 rows x 17 columns]
df1        DataFrame                 id          <...>\n[1499 rows x 4 columns]
df2        DataFrame                 id          <...>\n[1499 rows x 4 columns]
df3        DataFrame                  id         <...>\n[2000 rows x 4 columns]
df4        DataFrame                  id         <...>\n[2199 rows x 4 columns]
df_desc    DataFrame                  id         <...>\n[7197 rows x 4 columns]


### SQL query to list all tables

In [82]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())

[('Apps',), ('descriptionfile1',), ('descriptionfile2',), ('descriptionfile3',), ('descriptionfile4',)]


### Combine all description tables into 1 table SQL example:

#### Print the count of combined description table

+ cur.executescript is used to execute multiple lines os SQL
+ cur.execute will only execute 1 block of sql code

In [83]:
cur.executescript("""
DROP TABLE IF EXISTS Descriptionfile;                 
CREATE TABLE Descriptionfile AS 
                  SELECT * FROM descriptionfile1 
                  UNION 
                  SELECT * FROM descriptionfile2
                  UNION 
                  SELECT * FROM descriptionfile3
                  UNION 
                  SELECT * FROM descriptionfile4
                  ;

DROP TABLE IF EXISTS Control_DESC;                 

CREATE TABLE Control_DESC AS 
                 Select count(*) FROM Descriptionfile; 
""")

sql_desc_count = cur.execute("SELECT * FROM Control_DESC")
sql_desc_count = cur.fetchall()
print (f"Description File Table Count: {sql_desc_count}")

Description File Table Count: [(7197,)]


#### Print the count of apps table

In [87]:
sql_apps_count = cur.execute("SELECT count(*) FROM Apps")
print(f"Apps File Table Count: {eval('cur.fetchall()')}")


Apps File Table Count: [(7197,)]


### Print table columns:

In [97]:
cur.execute('SELECT * FROM Descriptionfile')
sql_desc_cols = [description[0] for description in cur.description]
print (f"Description SQL Table Columns: {sql_desc_cols}")
print('')
cur.execute('SELECT * FROM apps')
sql_apps_cols = [description[0] for description in cur.description]
print (f"APPS SQL Table Columns: {sql_apps_cols}")

Description SQL Table Columns: ['index', 'id', 'track_name', 'size_bytes', 'app_desc']

APPS SQL Table Columns: ['index', 'Unnamed: 0', 'id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices_num', 'ipadSc_urls_num', 'lang_num', 'vpp_lic']


In [ ]:
# code

## Modeling <a id='Modeling'></a>

#### Table for Average Rating by Language

In [111]:
cur.execute("""
CREATE TABLE IF NOT EXISTS Language_Avg_Rating as 
    SELECT 
        CASE 
        WHEN lang_num < 10 THEN '<10 languages'
        WHEN lang_num BETWEEN 10 and 30 THEN '10-30 languages'
        ELSE '>30 languages'
        END as language_ind,
        avg(user_rating) as Average_Rating
    FROM Apps
    GROUP BY language_ind;
""") 
cur.execute('SELECT * FROM Language_Avg_Rating')
Language_Avg_Rating_DF = cur.fetchall()
Language_Avg_Rating_DF
#print (f"Description File Table Count: {sql_desc_count}")


[('10-30 languages', 4.1305120910384066),
 ('<10 languages', 3.368327402135231),
 ('>30 languages', 3.7777777777777777)]

In [11]:
# code

In [ ]:
# code

In [ ]:
# code

## Evaluate <a id='Evaluate'></a>

In [ ]:
#code

In [16]:
conn.close()